In [51]:
import pandas as pd
import numpy as np
import os

df = pd.read_csv("cortical_metrics_1.4.csv")
area = 'VISp'

# Exc and PV have sufficient number of cells, so we'll filter out non-V1 Exc and PV.
# SST and VIP are small populations, so let's keep also non-V1 neurons
exclude = (df["cell_type"].isnull() | df["cell_type"].str.contains("EXC") | df["cell_type"].str.contains("PV")) \
        & (df["ecephys_structure_acronym"] != area)

df = df[~exclude]
print(f"Original: {df.shape[0]} cells,   filtered: {df.shape[0]} cells")

Original: 8799 cells,   filtered: 8799 cells


/tmp/ipykernel_454534/2068189933.py:5: DtypeWarning: Columns (60,62,65,67,69,72,119,123) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("cortical_metrics_1.4.csv")


In [52]:
# Count the number of observation from every session type
bo_counts = df[df["session_type"] == "brain_observatory_1.1"]["cell_type"].value_counts()
fc_counts = df[df["session_type"] == "functional_connectivity"]["cell_type"].value_counts()

# Some cells have very large values of RF. They are likely not-good fits, so ignore.
df.loc[(df["width_rf"] > 100), "width_rf"] = np.nan
df.loc[(df["height_rf"] > 100), "height_rf"] = np.nan

# define contrast response types
types = ["high", "band", "low"]
type_fracs = {}
type_fracs_err = {}
contrast_responsive = df["sig0.01"] == True
contrast_responsive_counts = df[contrast_responsive]["cell_type"].value_counts()
for t in types:
    type_counts = df[(df["best_model"] == t) & contrast_responsive]["cell_type"].value_counts()
    n = contrast_responsive_counts
    p = type_counts / n
    type_fracs[t] = p
    type_fracs_err[t] = np.sqrt(p * (1 - p) / n)

# Save the processed table
# df.to_csv("v1_OSI_DSI_DF.csv", sep=" ", index=False)

In [53]:
# do median and ignore non numerical columns
# df.groupby(["cell_type", "session_type"]).median(numeric_only=True)

In [54]:
# df[['cell_type','anterior_posterior_ccf_coordinate']].groupby("cell_type").median()

In [55]:
# set group statistics, and insert the cell counts for convenience
resource_tables = {}
resource_tables["median"] = df.groupby("cell_type").median(numeric_only=True)
resource_tables["mean"] = df.groupby("cell_type").mean(numeric_only=True)
resource_tables["sem"] = df.groupby("cell_type").sem(numeric_only=True)

for t in resource_tables.values():
    # insert counts and fractional elements that is common for all 3 tables
    t.insert(0, "n_cells_BO", bo_counts)
    t.insert(1, "n_cells_FC", fc_counts)
    t["n_cells_FC"][t["n_cells_FC"].isna()] = 0
    t["n_cells_FC"] = t["n_cells_FC"].astype(dtype=int)

    # t.insert(-1, "frac_highpass", type_fracs["high"])
    t["frac_highpass"] = type_fracs["high"]
    t["frac_bandpass"] = type_fracs["band"]
    t["frac_lowpass"] = type_fracs["low"]

# sem is a special case. re-insert the estimated error of the fraction
resource_tables["sem"]["frac_highpass"] = type_fracs_err["high"]
resource_tables["sem"]["frac_bandpass"] = type_fracs_err["band"]
resource_tables["sem"]["frac_lowpass"] = type_fracs_err["low"]

/tmp/ipykernel_454534/3857521699.py:11: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  t["n_cells_FC"][t["n_cells_FC"].isna()] = 0
/tmp/ipykernel_454534/3857521699.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [56]:
selected_metrics = [
    "n_cells_BO",
    "n_cells_FC",
    "g_osi_dg",
    "g_dsi_dg",
    "pref_sf_sg",
    "pref_tf_dg",
    "firing_rate_sp",
    "firing_rate_rf",
    "firing_rate_dg",
    "width_rf",
    "height_rf",
    "fano_dg",
    "frac_highpass",
    "frac_bandpass",
    "frac_lowpass",
]

# For the full list of metrics, refer to list(df.columns)
#
# See https://allensdk.readthedocs.io/en/latest/visual_coding_neuropixels.html
# for detailed descriptions of the metrics.

In [57]:
pd.set_option("display.float_format", "{:0.2f}".format)
print('Median functional responses')
resource_tables["median"][selected_metrics]

Median functional responses


,n_cells_BO,n_cells_FC,g_osi_dg,g_dsi_dg,pref_sf_sg,pref_tf_dg,firing_rate_sp,firing_rate_rf,firing_rate_dg,width_rf,height_rf,fano_dg,frac_highpass,frac_bandpass,frac_lowpass
cell_type,,,,,,,,,,,,,,,
ALL_L1,50,91,0.18,0.11,0.04,2.00,2.21,0.00,0.40,25.90,28.65,3.37,0.83,0.17,NaN
EXC_L23,203,300,0.30,0.19,0.04,2.00,0.63,0.29,0.66,13.05,14.68,2.73,0.54,0.29,0.18
EXC_L4,301,328,0.20,0.12,0.04,2.00,2.12,1.01,2.05,25.28,28.29,3.31,0.61,0.30,0.09
EXC_L5,482,493,0.19,0.08,0.04,2.00,4.05,2.41,4.47,32.85,37.05,3.14,0.71,0.23,0.06
EXC_L6,191,153,0.42,0.08,0.04,2.00,1.86,1.31,2.71,31.96,34.75,3.34,0.76,0.18,0.06
PV_L23,75,58,0.11,0.09,0.08,2.00,7.35,3.38,5.20,17.98,21.95,6.20,0.67,0.17,0.17
PV_L4,98,81,0.11,0.07,0.04,2.00,8.86,4.34,7.72,28.14,28.83,6.56,0.62,0.22,0.16
PV_L5,99,101,0.12,0.06,0.08,2.00,8.95,4.78,10.25,31.78,34.51,6.59,0.79,0.15,0.06
PV_L6,56,34,0.16,0.06,0.06,2.00,9.76,5.27,11.86,46.37,48.53,5.59,0.81,0.15,0.04
